In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from utils import *
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
plt.rcParams['font.family'] = 'Helvetica'

description =(
'''Take a model with very small size (4,8,8,2). It will function as the *base model*. Only considering weights for pruning, this model has 112 p-params.
With a fixed p-rate of 0.32, a larger model architecture can be calculated, which would have the same number of p-parameters after pruning with (roughly) the p-rate once.
This process is termed "model extension". A model is extended by n, where n is the number of p-levels needed to get it to the size of the base model. n is termed the "extension-level".
The model architectures are restricted to having the same number of hidden layers, and the same number of hidden neurons in each hidden layer.
Therefore, the exact p-rate differs slightly, depending on the extension level.
'''
)

In [ ]:
sweep_ids=['wpcowdl5', 'j9lvvxjg', 'a5kr3muw']

try: print('Histories already loaded.') if hc else None
except: hc = load_histories_and_configs(sweep_ids)

# define dataframes to concat
_split = make_split_df(hc)
_levels = make_df(hc, 'extension_levels')
_shape = make_df(hc, 'model_shape').map(lambda shape: shape[1])
_p_params_begin = make_df(hc, 'param_trajectory').map(lambda x: x[0])
#_shape['hidden_dim'] = _shape.map(lambda shape: shape[1])
_splitrange = make_splitrange_df(hc)
_first, _last = make_dfs_at_splitrange(hc, _splitrange)

In [ ]:
plt.figure(figsize=(6,4))
valloss = make_df_from_history(hc, 'val-loss', invert=False, drop_first=True)
active = make_df_from_history(hc, 'active-weights-abs', invert=False, drop_first=True)
potential = make_df_from_history(hc, 'untapped-potential', invert=False, drop_first=True)

mean_active = pd.concat([_levels, active], axis=1).groupby('extension_levels').mean()
mean_loss = pd.concat([_levels, valloss], axis=1).groupby('extension_levels').mean()

# find best loss for each extension level (row)
Y = mean_loss.to_numpy()
X = mean_active.to_numpy()
indices = np.nanargmin(Y, axis=1)
rows = np.arange(indices.shape[0])

y = Y[rows, indices]
x = X[rows, indices]

cmap_blue = LinearSegmentedColormap.from_list('', ['#1E88E5', '#3c1ee5'], N=len(rows))
plt.scatter(x,y, color='blue', label='best', alpha=0.6)
#plt.scatter(x,y, c=rows, cmap='winter', label='best overall')
#plt.colorbar()

active = (potential == 0) * active
active[active == 0] = np.nan
valloss = (potential == 0) * valloss
valloss[valloss == 0] = np.nan
mean_active = pd.concat([_levels, active], axis=1).groupby('extension_levels').mean()
mean_loss = pd.concat([_levels, valloss], axis=1).groupby('extension_levels').mean()

# find best loss for each extension level (row)
Y = mean_loss.to_numpy()
X = mean_active.to_numpy()

mask = np.all(np.isnan(X), axis=1)
X, Y = X[~mask], Y[~mask]

indices = np.nanargmin(Y, axis=1)
rows = np.arange(indices.shape[0])

y = Y[rows, indices]
x = X[rows, indices]
#plt.scatter(x,y, c=rows, cmap='plasma')
cmap_red = LinearSegmentedColormap.from_list('', ['#D81B60', '#a60c0c'], N=len(rows))
#plt.scatter(x,y, c=rows, cmap='spring')
#plt.scatter(x,y, c=rows, cmap='spring')
plt.scatter(x,y, color='red', label='best split', marker='^')
plt.ylim([0, 0.0125])
plt.legend()
#plt.colorbar()
plt.ylabel('validation loss')
plt.xlabel('number of weights active')
plt.savefig('results.svg')

In [ ]:
plt.figure(figsize=(6,4))
valloss = make_df_from_history(hc, 'val-loss', invert=False, drop_first=True)
active = make_df_from_history(hc, 'active-weights-abs', invert=False, drop_first=True)
potential = make_df_from_history(hc, 'untapped-potential', invert=False, drop_first=True)

mean_active = pd.concat([_levels, active], axis=1).groupby('extension_levels').mean()
mean_loss = pd.concat([_levels, valloss], axis=1).groupby('extension_levels').mean()

# find best loss for each extension level (row)
Y = mean_loss.to_numpy()
X = mean_active.to_numpy()
indices = np.nanargmin(Y, axis=1)
rows = np.arange(indices.shape[0])

y = Y[rows, indices]
x = X[rows, indices]

cmap_blue = LinearSegmentedColormap.from_list('', ['#1E88E5', '#3c1ee5'], N=len(rows))
#plt.scatter(x,y, color='blue', label='best', alpha=0.6)
#plt.scatter(x,y, c=rows, cmap='winter', label='best overall')
#plt.colorbar()

active = (potential == 0) * active
active[active == 0] = np.nan
valloss = (potential == 0) * valloss
valloss[valloss == 0] = np.nan
mean_active = pd.concat([_levels, active], axis=1).groupby('extension_levels').mean()
mean_loss = pd.concat([_levels, valloss], axis=1).groupby('extension_levels').mean()

# find best loss for each extension level (row)
Y = mean_loss.to_numpy()
X = mean_active.to_numpy()

mask = np.all(np.isnan(X), axis=1)
X, Y = X[~mask], Y[~mask]

indices = np.nanargmin(Y, axis=1)
rows = np.arange(indices.shape[0])

y = Y[rows, indices]
x = X[rows, indices]
#plt.scatter(x,y, c=rows, cmap='plasma')
#cmap_red = LinearSegmentedColormap.from_list('', ['#D81B60', '#a60c0c'], N=len(rows))
#plt.scatter(x,y, c=rows, cmap='spring')
plt.scatter(x,y, c=rows, cmap='winter')
#plt.scatter(x,y, color='red', label='best split', marker='^')
#plt.ylim([0, 0.0125])
plt.legend()
plt.colorbar()
plt.ylabel('validation loss')
plt.xlabel('number of weights active')
plt.savefig('results.svg')

In [ ]:

Y = mean_active.to_numpy()
plt.imshow(Y)

In [ ]:

for id in range(1,len(df_x.index)):
    plt.scatter(df_x[id],df_y[id], c=df_x.index, cmap='plasma')
    plt.colorbar() if id ==1 else None

In [ ]:

cmap = plt.cm.get_cmap('PiYG', len(df_x.columns))  # Using 'viridis' colormap
#plt.figure(figsize=(10, 6))
fig, ax = plt.subplots()
for i, column in enumerate(df_x.columns):
    color = cmap(i)
    cax = ax.scatter(df_x[column], df_y[column], c=[i]*len(df_y[column]), cmap=cmap, label=column)

fig.colorbar(cax)
plt.yscale('log')

In [ ]:
df_x

In [ ]:
_active_w = make_df_from_history(hc, 'active-weights-rel', invert=True, drop_first=True)
_loss = make_df_from_history(hc, 'val-loss', invert=True, drop_first=True)

df_loss = pd.concat([_levels, _loss], axis=1).groupby('extension_levels').mean()
df_active_weights = pd.concat([_levels, _active_w], axis=1).groupby('extension_levels').mean()

_active_w_abs = make_df_from_history(hc, 'active-weights-abs', invert=True, drop_first=True)
df_active_weights_abs = pd.concat([_levels, _active_w_abs], axis=1).groupby('extension_levels').mean()

# Hypothesis 
## 'Collateral Damage':
for each p-level, there is 'collateral damage'. Some p-parameters are made inactive or zombious. -> the relative amount of active p-parameters decreases, the more p-levels there are.
### How to test
plot the relative amount of at each level, from each run. If it is true, each run with more levels should have less active p-parameters at the corresponding extension level. Because the networks have the same number of p-parameters at the corresponding levels the comparisson should be fair.

Indeed, the more pruning iterations, generally decrease the percentage of active-weights in the network.

In [ ]:
df_active = make_df_from_history(hc, 'active-weights-rel', invert=True, drop_first=True)
df = pd.concat([_shape, df_active], axis=1).groupby('model_shape').mean().reset_index()

x = df['model_shape'].to_list()
y = df[1].to_list()

xticks = x
xticks = xticks[0:1] + xticks[8:9] + xticks[14:15] + xticks[18:]
xticklabels = xticks


plt.scatter(x,y, label='% active of available weights')
plt.xticks(xticks, xticklabels)
plt.xlabel('hidden neurons per layer')
plt.ylabel('% weights remaining')
plt.legend()
plt.savefig('collateral-damage.svg')
plt.show()

In [ ]:
df_active_weights

In [ ]:
plt.plot(df_active_weights.T)

In [ ]:
indices = [0, 6, 12, 18, 24]
plt.plot(df_active_weights.iloc[indices].T, marker='o')
plt.legend()
plt.show()



### Peak Performance
The networks seem to reach peak performance around the same number of p-parameters

In [ ]:
indices = df_active_weights_abs.index

for i in indices:
    x = df_active_weights_abs.loc[i]
    y = df_loss.loc[i]
    plt.plot(x,y, marker='', label=f'e={i}', alpha=0.4)
    
plt.xscale('log'), plt.yscale('log')
# plt.legend()
plt.show()

### When do networks split
and when do they degrade?
Why do networks only start to degrade after 12 extension levels, and continue to do so when extending further?
Lets look at the number of active parameters at the time they split and at the time they degrade.

Do they split earlier, in terms of iteration and in terms of active weights?

Results show some signs that tey split in earlier iterations, but not clearly. Could be noise as well.
However, the more e-levels they have, the less active parameters the have when they split.

In [ ]:
group_df = _levels
source_df = _first
source_keys = [
    'active-weights-abs', 
    'active-features-rel',  
    'active-weights-rel',
    'zombious-abs',
    ]

group = make_group(group_df, source_df, source_keys)

for key in source_keys:
    df = mean_std_from_group(group, key)
    errorbar_from_df(df, label=key, marker='o', alpha=0.6)

Do they degrade earlier?
They clearly degrade with more p-parameters available, meaning they degrade in earlier iterations. However, the number of active parameters stays about the same, independent of the number of e-levels.

In [ ]:
group_df = _shape
source_df = _first
source_keys = ['pparams', 'active-weights-abs']

group = make_group(group_df, source_df, source_keys)
plt.figure(figsize=(10, 3)) 

df = mean_std_from_group(group, 'active-weights-abs').reset_index()
plt.errorbar(
    x=df['xticks'], 
    y=df['y'], 
    yerr=df['yerr'],
    alpha=0.6,
    marker='o',
    label='active weights'
)
df = mean_std_from_group(group, 'pparams').reset_index()
plt.errorbar(
    x=df['xticks'], 
    y=df['y'], 
    yerr=df['yerr'],
    alpha=0.6,
    marker='o',
    label='available weights'
)

plt.title('Extending the model')
# x axis data
xticks = np.arange(len(df.index))
xticklabels = [f'{l}\n{pr}' for l, pr in df['model_shape'].items()]

plt.xticks(xticks, xticklabels)
plt.xlabel('extension level\nhidden dim', loc='left')
plt.ylabel('number of parameters')
plt.yscale('log')
plt.legend()
plt.grid(which='minor')
plt.yticks(minor=True)
plt.tight_layout()
plt.savefig("extending-active-available.svg")

In [ ]:
active = pd.concat([_levels, _first['active-weights-abs']],axis=1).groupby('extension_levels').mean()
plt.plot(active, label='active', marker='o', alpha=0.6)

#active_features = pd.concat([_levels, _first['active-features-abs']],axis=1).groupby('extension_levels').mean()
#plt.plot(active_features, label='features', marker='o', alpha=0.6)

pparams = pd.concat([_levels, _first['pparams']],axis=1).groupby('extension_levels').mean()
plt.plot(pparams, label='available', marker='o', alpha=0.6)

plt.xticks(active.index)
plt.legend()
plt.yscale('log')
plt.show()

when Looking at weights and biases together, the Split networks actually are larger when increasing the e-level. this is mainly due to more biases in the network.

In [ ]:
fz = pd.concat([_levels, _first['active-abs']],axis=1).groupby('extension_levels').mean()
plt.plot(fz, label='active parameters', marker='o', alpha=0.6)

lz = pd.concat([_levels, _last['active-abs']],axis=1).groupby('extension_levels').mean()
plt.plot(lz, label='active parameters', marker='o', alpha=0.6)

plt.xticks(active.index)
plt.legend()
plt.show()

# Can this be true?
That there are 7 times the number of biases than weights ACTIVE in the network?

In [ ]:
_first['active-features-abs'] / _first['active-weights-abs']
fz = pd.concat([_levels, _first['active-features-abs'] / _first['active-weights-abs']],axis=1).groupby('extension_levels').mean()
lz = pd.concat([_levels, _last['active-features-abs'] / _last['active-weights-abs']],axis=1).groupby('extension_levels').mean()
plt.plot(lz, label='last weights / biases ratio', marker='o', alpha=0.6)
plt.plot(fz, label='first weights / biases ratio', marker='o', alpha=0.6)

plt.xticks(active.index)
plt.legend()
plt.show()

In [ ]:
yfirst = pd.concat([_levels, _last['active-weights-rel']],axis=1).groupby('extension_levels').mean()
ylast = pd.concat([_levels, _first['active-weights-rel']],axis=1).groupby('extension_levels').mean()

plt.plot(yfirst, label='first', marker='o', alpha=0.6)
plt.plot(ylast, label='last', marker='o', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
# Plotting a heatmap with a logarithmic colorbar using matplotlib
plt.figure(figsize=(8, 6))
plt.imshow(data, cmap='tab20', aspect='auto', norm=LogNorm())
plt.colorbar(label='Logarithmic Value')
plt.xticks(ticks=range(data.shape[1]), labels=data.columns)
plt.yticks(ticks=range(data.shape[0]), labels=data.index)
plt.title("Heatmap of DataFrame (Logarithmic Scale)")
plt.show()

In [ ]:
title='Collateral Damage'

w_active = pd.DataFrame([h['active-weights-abs'] for h,_ in hc]).reset_index().drop(0, axis=1)
df = pd.concat([_levels,w_active], axis=1).sort_values('extension_levels')

In [ ]:
title='Extending the base model'

ydf = pd.concat([_split, _levels], axis=1).groupby('extension_levels').sum()
xdf = pd.concat([_shape, _levels], axis=1).drop_duplicates()
xdf['xticklabels'] = xdf.apply(lambda df: f"{df['extension_levels']}\n{df['model_shape']}", axis=1)
xdf = xdf.sort_values('extension_levels').reset_index(drop=True)
xdf['xticks'] = xdf.index

fig, axes = plt.subplots(nrows=2, figsize=(16,9), sharey=True, sharex=False)
fig.suptitle(title, fontweight='regular', fontsize=15, fontname='Helvetica')

# plot the data
ydf[splitters.keys()].plot.bar(
    ax=axes[0],
    width=0.95,  
    color=splitters.values(),
    rot=0,
    stacked=True
)
ydf[nonsplitters.keys()].plot.bar(
    ax=axes[1],
    width=0.95,  
    color=nonsplitters.values(),
    rot=0,
    stacked=True
)

for ax in axes:
    ax.set_yticks(np.arange(np.max(ydf)+1))
    ax.set_xticks(xdf['xticks'])
    ax.set_xticklabels(xdf['xticklabels'], fontsize=10, color='gray')
    ax.set_ylabel('num models', fontsize=14)
    ax.set_xlabel('extensions\nhidden-dim', fontsize=10, color='gray')
    ax.xaxis.set_label_coords(0.0, -0.024)

plt.tight_layout(
    #rect=[0, 0.03, 1, 0.95]
)
set_style(axes)
plt.savefig("hist.svg")